In [ ]:
from __future__ import print_function, division
import numpy as np
import scipy as sp
from numba import jit

from dstrf import mat, strf

# plotting packages
%matplotlib inline
import matplotlib.pyplot as plt # plotting functions
import seaborn as sns           # data visualization package
sns.set_style("whitegrid")

In [ ]:
# model parameters: (α1, α2, ω, τ1, τ2, tref)
matparams = np.asarray([100, 2, 7, 10, 200, 2], dtype='d')
model_dt = 0.5

# data parameters
duration = 500000
n_samples = int(duration / model_dt)
n_assim = 1
n_test = 0

In [ ]:
# convolution - simple alpha kernel
from scipy.signal import resample
stim_dt = 10.0
upsample = int(stim_dt / model_dt)

# alpha filter
tau_h = 50
tt = np.arange(0, 600, stim_dt)
ka = np.flipud(tt / tau_h * np.exp(-tt / tau_h))

# difference of gammas:
from scipy.special import gamma
tau_h1 = 600. / 32
tau_h2 = 600. / 16
kg1 = 1/(gamma(6)*tau_h1)*(tt/tau_h1)**5 * np.exp(-tt/tau_h1)
kg2 = 1/(gamma(6)*tau_h2)*(tt/tau_h2)**5 * np.exp(-tt/tau_h2)
kg = np.flipud(kg1 - kg2 / 1.5)
kg /= np.linalg.norm(kg)

# select a filter
k1 = (kg * 5)[::-1]
plt.plot(k1)

def filter_stimulus(S, k1, upsample=None):
    X = np.convolve(S, k1, mode="full")[:S.size]
    if upsample is not None:
        t = np.arange(X.size)
        tu = np.arange(0, X.size, 1 / upsample)
        return np.interp(tu, t, X)
    else:
        return X
    

In [ ]:
@jit
def predict_spikes(V, params, dt, upsample):
    N = V.size * upsample
    S = np.zeros(N, dtype='i')
    Vi = np.zeros(N, dtype='d')
    R = np.random.uniform(size=N)
    omega = params[2]
    A1 = np.exp(-dt / params[3])
    A2 = np.exp(-dt / params[4])
    i_refrac = int(params[5] / dt)
    H1 = H2 = 0
    iref = 0
    for i in range(N):
        Vi[i] = V[i // upsample]
        H1 *= A1
        H2 *= A2
        p = np.exp(Vi[i] - H1 - H2 - omega) * dt
        if i > iref and p > R[i]:
            H1 += params[0]
            H2 += params[1]
            iref = i + i_refrac
            S[i] = 1
    return Vi, S


def lci_poisson(V, H, spikes, params, dt):
    mu = V - H[:, 0] * params[0] - H[:, 1] * params[1] - params[2]
    lp = spikes * mu - dt * np.exp(mu)
    return lp.sum()


In [ ]:
# generate data to fit
np.random.seed(1)
data = []
stim = np.random.randn(int(n_samples / (stim_dt / model_dt)))
stim[:100] = 0
        
V = filter_stimulus(stim, k1, upsample=1)
for i in range(n_assim + n_test):
    Vinterp, spikes = predict_spikes(V, matparams, model_dt, upsample)
    H = np.c_[mat.predict_adaptation(spikes, matparams[3], model_dt),
              mat.predict_adaptation(spikes, matparams[4], model_dt)]
    z = np.nonzero(spikes)[0]
    d = {"V": Vinterp,
         "H": H,
         "duration": duration,
         "spike_t": z, 
         "spike_v": spikes,
         "lci": lci_poisson(Vinterp, H, spikes, matparams, model_dt)
         }
    data.append(d)

# split into assimilation and test sets
assim_data = data[:n_assim]
test_data = data[n_assim:]

In [ ]:
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)
for i, d in enumerate(data):
    ax2.vlines(d["spike_t"], i, i + 0.5)
ax1.plot(d["V"])
ax1.set_xlim(0, 8000)
ax2.set_xlim(0, 8000)
print(sum(d["lci"] for d in data))
print("spikes: {}; rate: {} / dt".format(np.mean([d["spike_t"].size for d in data]), 
                                         np.mean([d["spike_t"].size / d["duration"] for d in data])))

In [ ]:
# initial guess of strf
# bin the spike train down to the stimulus rate
psth = np.sum([np.sum(d["spike_v"].reshape((int(n_samples / upsample), upsample)), axis=1) for d in data], axis=0)
# generate design matrix for stimulus
X_stim = strf.design_matrix(stim.reshape(1, stim.size), k1.size)
w0 = np.dot(X_stim.T, psth) / np.sum(psth)
rf_est = w0[::-1]
plt.plot(rf_est)
plt.plot(k1)

In [ ]:
# save data for import into matlab
import scipy.io as sio
d = data[0]
Itot = d["V"] - d["H"][:, 0] * matparams[0] - d["H"][:, 1] * matparams[1] - matparams[2]
sio.savemat("glmat_1d_data.mat", {"spikes": d["spike_v"],
                                  "spike_dt": model_dt,
                                  "stim_dt": stim_dt,
                                  "stim": stim,
                                  "rf": k1,
                                  "Itot": Itot,
                                  "Istim": d["V"],
                                  "matparams": matparams})

In [ ]:
# evaluate cost function with matrix algebra
import scipy.sparse as sps
dc = matparams[2]
alpha = matparams[:2]
k = k1[::-1]
w0 = np.r_[dc, alpha, k]
spk = d["spike_t"]
interp = sps.kron(sps.eye(stim.size), np.ones((upsample, 1), dtype='i'), format='csc')
interpsp = interp[spk,:]
V = interp.dot(np.dot(X_stim, k)) - np.dot(H, alpha) - dc
# log likelihood function
rr = np.exp(V)
ll = rr.sum() * model_dt - V[spk].sum()
# gradient of ll:
dLdk0 = np.dot(rr, interp.dot(X_stim))  # non-spiking term
dLdb0 = rr.sum()
dLda0 = np.dot(rr, H)
dLdk1 = interpsp.dot(X_stim).sum(0)     # spiking term
dLdb1 = spk.size
dLda1 = H[spk].sum(0)
gradL = np.r_[dLdb0 * model_dt - dLdb1, dLda0 * model_dt - dLda1, dLdk0 * model_dt - dLdk1]
ll, gradL

In [ ]:
# now do it with theano
import theano
import theano.tensor as T
from theano import sparse

M = theano.shared(interp)
dt = theano.shared(model_dt)
Xstim = theano.shared(X_stim)
Xspke = theano.shared(H)
sidx = theano.shared(spk)

w = T.dvector('w')
dc = w[0]
alpha = w[1:3]
k = w[3:]
v = T.dvector('v')
Vx = T.dot(Xstim, k)
Hx = T.dot(Xspke, alpha)
Vi = sparse.structured_dot(M, Vx.reshape((Vx.size, 1))).squeeze() - Hx - dc
ll = T.exp(Vi).sum() * dt - Vi[sidx].sum() 
dL = T.grad(ll, w)
ddL = theano.gradient.hessian(ll, w)

th_Vi = theano.function([w], Vi)
th_loglike = theano.function([w], ll, on_unused_input='warn')
th_grad = theano.function([w], dL, on_unused_input='warn')
th_hess = theano.function([w], ddL)

In [ ]:
(th_loglike(w0),
 th_grad(w0))

In [ ]:
import scipy.optimize as op
w1 = op.fmin_ncg(th_loglike, w0, th_grad, fhess=th_hess, maxiter=100)

In [ ]:
print(w1[:3])
rf_est = w1[:3:-1]
plt.plot(rf_est)
plt.plot(w0[:3:-1])
plt.plot(k1)